### You can also download the model from Hugging Face

Hugging Face Model Repo:
https://huggingface.co/JesseHuang922/lv_spaCy_CNN



In [5]:
# ==============================
# Cell 0：Import and directories
# ==============================
from pathlib import Path
import os
import spacy
from spacy.lookups import Lookups
from spacy.tokens import DocBin
from spacy.cli.package import package

# Create root directory
project_root = Path(".").resolve()

# Project structure
corpus_dir = project_root / "corpus"
models_dir = project_root / "models"
model_name = "lv_spacy_cnn"
trained_model_path = models_dir / model_name / "model-best"
final_model_path = models_dir / model_name / "model_lv_cnn"
lookups_path = project_root / "lookups_lv"
package_output_dir = project_root / "packages"
config_path = project_root / "config" / "config_spacy_cnn.cfg"

# Create directories
for p in [corpus_dir, models_dir, models_dir / model_name, package_output_dir, lookups_path, project_root / "config"]:
    p.mkdir(parents=True, exist_ok=True)

print("Project structure has been set!")



Project structure has been set!


In [6]:
# ================================================
# Cell 1：Convert all conllu files to spaCy format
# ================================================
!python -m spacy convert ud_latvian/lv_lvtb-ud-train.conllu ./corpus -n 10
!python -m spacy convert ud_latvian/lv_lvtb-ud-dev.conllu ./corpus -n 10
!python -m spacy convert ud_latvian/lv_lvtb-ud-test.conllu ./corpus -n 10


ℹ Grouping every 10 sentences into a document.
✔ Generated output file (1506 documents):
corpus/lv_lvtb-ud-train.spacy
ℹ Grouping every 10 sentences into a document.
✔ Generated output file (208 documents):
corpus/lv_lvtb-ud-dev.spacy
ℹ Grouping every 10 sentences into a document.
✔ Generated output file (240 documents):
corpus/lv_lvtb-ud-test.spacy


In [7]:
# ===================================================================================================================
# Cell 2：Initializing config (if you add -gpu, the backbone would be changed to transformer based pre-trained model)
# ===================================================================================================================
!python -m spacy init config ./config/config_spacy_cnn.cfg \
    --lang lv \
    --pipeline tok2vec,tagger,morphologizer,parser,senter \
    --optimize efficiency



ℹ Generated config template specific for your use case
- Language: lv
- Pipeline: tagger, morphologizer, parser, senter
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config/config_spacy_cnn.cfg
You can now add your data and train your pipeline:
python -m spacy train config_spacy_cnn.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [8]:
# ==========================
# Cell 3：Modify config
# ==========================

# Read config file
cfg_text = config_path.read_text(encoding="utf-8")

# Replace the training/validation set path
cfg_text = cfg_text.replace("train = null", f"train = {corpus_dir}/lv_lvtb-ud-train.spacy")
cfg_text = cfg_text.replace("dev = null", f"dev = {corpus_dir}/lv_lvtb-ud-dev.spacy")

# # Turn on Mixed Precision
cfg_text = cfg_text.replace("mixed_precision = false", "mixed_precision = true")

config_path.write_text(cfg_text, encoding="utf-8")
print("config updated!")


config updated!


In [9]:
# ====================================
# Cell 4: Generate lemma lookup table
# ====================================

# Load training set
docbin = DocBin().from_disk(corpus_dir / "lv_lvtb-ud-train.spacy")
lemma_dict = {}

for doc in docbin.get_docs(spacy.blank("lv").vocab):
    for token in doc:
        if token.lemma_:
            lemma_dict[token.text.lower()] = token.lemma_

lookups = Lookups()
lookups.add_table("lemma_lookup", lemma_dict)
lookups.to_disk(lookups_path)
print(f"Generated lemma lookup table, dir: {lookups_path}")


/home/jesse/Projects/myenvs/spacy_lv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Generated lemma lookup table, dir: /home/jesse/Projects/myprojs/LV_spaCy_Pipeline/LV_spaCy_CNN/lookups_lv


In [10]:
# ==================================
# Cell 5：Train spaCy tok2vec model
# ==================================
!python -m spacy train ./config/config_spacy_cnn.cfg \
    --output ./models/lv_spacy_cnn\
    --paths.train ./corpus/lv_lvtb-ud-train.spacy \
    --paths.dev ./corpus/lv_lvtb-ud-dev.spacy \
    --gpu-id 0


ℹ Saving to output directory: models/lv_spacy_cnn
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'tagger', 'morphologizer', 'parser',
'senter']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS TAGGER  LOSS MORPH...  LOSS PARSER  LOSS SENTER  TAG_ACC  POS_ACC  MORPH_ACC  DEP_UAS  DEP_LAS  SENTS_P  SENTS_R  SENTS_F  SCORE 
---  ------  ------------  -----------  -------------  -----------  -----------  -------  -------  ---------  -------  -------  -------  -------  -------  ------
  0       0          0.00       202.94         202.86       447.71       112.50    18.53    31.64      21.10     8.67     6.82     0.01     0.19     0.02    0.13
  0     200       3310.52     23151.48       21819.23     33169.44      2844.51    51.81    77.87      58.19    50.06    32.78    47.32    62.74    53.95    0.54
  0    

In [11]:
# ========================
# Cell 6: Evaluation
# ========================
!python -m spacy evaluate ./models/lv_spacy_cnn//model-best ./corpus/lv_lvtb-ud-test.spacy --gpu-id 0

ℹ Using GPU: 0

================================== Results ==================================

TOK      99.53
TAG      86.64
POS      94.79
MORPH    90.77
UAS      81.04
LAS      75.46
SENT P   96.84
SENT R   95.83
SENT F   96.33
SPEED    23640


============================== MORPH (per feat) ==============================

                P       R       F
ExtPos      89.92   83.45   86.57
Case        92.03   91.63   91.83
Gender      93.61   93.10   93.35
Number      94.13   93.18   93.65
Person      95.35   94.22   94.78
PronType    98.42   98.00   98.21
Evident     94.52   93.35   93.93
Mood        93.70   92.30   92.99
Polarity    94.03   93.07   93.55
Tense       91.08   89.68   90.38
VerbForm    92.68   91.72   92.20
Voice       93.64   92.00   92.81
Definite    90.28   88.17   89.21
Degree      92.72   90.81   91.76
Poss       100.00   99.45   99.73
NumType     98.85   77.21   86.70
Reflex      93.89   92.58   93.23
Aspect      87.88   85.95   86.90
Foreign     84.68   49.47  

In [12]:
# ==========================================
# Cell 7：Add Lemmatizer (lookup) to model
# ==========================================
import spacy
from spacy.lookups import Lookups
from pathlib import Path

trained_model_path = "./models/lv_spacy_cnn/model-best"
final_model_path = "./models/lv_spacy_cnn/model_lv_cnn"
lookups_path = "./lookups_lv"

# Load trained model
nlp = spacy.load(trained_model_path)

# Load lookups
lookups = Lookups().from_disk(lookups_path)

# Add lemmatizer（lookups) 
lemmatizer = nlp.add_pipe("lemmatizer", config={"mode":"lookup"}, last=True)
lemmatizer.lookups = lookups  

# Sve model
nlp.to_disk(final_model_path)
print(f"Model saved to {final_model_path}, with lemmatizer + lookups")


Model saved to ./models/lv_spacy_cnn/model_lv_cnn, with lemmatizer + lookups


In [13]:
# =======================
# Cell 8：Packaging
# =======================

from spacy.cli.package import package
from pathlib import Path
import os

project_root = Path(".").resolve()
os.environ["PYTHONPATH"] = f"{project_root}:{os.environ.get('PYTHONPATH','')}"

# Packaging
package(
    input_dir=Path(final_model_path),
    output_dir=Path(package_output_dir),
    name="spacy_cnn",
    version="1.0.0",
    force=True
)

print(f"Finished, packaged model can be found here: {package_output_dir}")

* Creating isolated environment: venv+pip...
* Installing packages in isolated environment:
  - setuptools >= 40.8.0
* Getting build dependencies for sdist...
running egg_info
creating lv_spacy_cnn.egg-info
writing lv_spacy_cnn.egg-info/PKG-INFO
writing dependency_links to lv_spacy_cnn.egg-info/dependency_links.txt
writing entry points to lv_spacy_cnn.egg-info/entry_points.txt
writing top-level names to lv_spacy_cnn.egg-info/top_level.txt
writing manifest file 'lv_spacy_cnn.egg-info/SOURCES.txt'
reading manifest file 'lv_spacy_cnn.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'lv_spacy_cnn.egg-info/SOURCES.txt'
* Building sdist...


running sdist
running egg_info
writing lv_spacy_cnn.egg-info/PKG-INFO
writing dependency_links to lv_spacy_cnn.egg-info/dependency_links.txt
writing entry points to lv_spacy_cnn.egg-info/entry_points.txt
writing top-level names to lv_spacy_cnn.egg-info/top_level.txt
reading manifest file 'lv_spacy_cnn.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'lv_spacy_cnn.egg-info/SOURCES.txt'
running check
creating lv_spacy_cnn-1.0.0
creating lv_spacy_cnn-1.0.0/lv_spacy_cnn
creating lv_spacy_cnn-1.0.0/lv_spacy_cnn.egg-info
creating lv_spacy_cnn-1.0.0/lv_spacy_cnn/lv_spacy_cnn-1.0.0
creating lv_spacy_cnn-1.0.0/lv_spacy_cnn/lv_spacy_cnn-1.0.0/lemmatizer/lookups
creating lv_spacy_cnn-1.0.0/lv_spacy_cnn/lv_spacy_cnn-1.0.0/morphologizer
creating lv_spacy_cnn-1.0.0/lv_spacy_cnn/lv_spacy_cnn-1.0.0/parser
creating lv_spacy_cnn-1.0.0/lv_spacy_cnn/lv_spacy_cnn-1.0.0/senter
creating lv_spacy_cnn-1.0.0/lv_spacy_cnn/lv_spacy_cnn-1.0.0/tagger
creating lv_spacy_cnn-1.0.0/lv

In [14]:
# ===========================
# Cell 9：Build wheel + sdist
# ===========================
import subprocess
from pathlib import Path

package_output_dir = Path("./packages/lv_spacy_cnn-1.0.0")

# build wheel and sdist
subprocess.run(
    ["python", "-m", "build", "--wheel", "--sdist"],
    cwd=str(package_output_dir)
)

print(f"wheel + sdist are built, dir: {package_output_dir / 'dist'}")


* Creating isolated environment: venv+pip...
* Installing packages in isolated environment:
  - setuptools >= 40.8.0
* Getting build dependencies for wheel...
running egg_info
writing lv_spacy_cnn.egg-info/PKG-INFO
writing dependency_links to lv_spacy_cnn.egg-info/dependency_links.txt
writing entry points to lv_spacy_cnn.egg-info/entry_points.txt
writing top-level names to lv_spacy_cnn.egg-info/top_level.txt
reading manifest file 'lv_spacy_cnn.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'lv_spacy_cnn.egg-info/SOURCES.txt'
* Building wheel...


running bdist_wheel
running build
running build_py
creating build/lib/lv_spacy_cnn
copying lv_spacy_cnn/__init__.py -> build/lib/lv_spacy_cnn
creating build/lib/lv_spacy_cnn/lv_spacy_cnn-1.0.0
copying lv_spacy_cnn/lv_spacy_cnn-1.0.0/meta.json -> build/lib/lv_spacy_cnn/lv_spacy_cnn-1.0.0
copying lv_spacy_cnn/lv_spacy_cnn-1.0.0/README.md -> build/lib/lv_spacy_cnn/lv_spacy_cnn-1.0.0
copying lv_spacy_cnn/lv_spacy_cnn-1.0.0/tokenizer -> build/lib/lv_spacy_cnn/lv_spacy_cnn-1.0.0
copying lv_spacy_cnn/lv_spacy_cnn-1.0.0/config.cfg -> build/lib/lv_spacy_cnn/lv_spacy_cnn-1.0.0
creating build/lib/lv_spacy_cnn/lv_spacy_cnn-1.0.0/morphologizer
copying lv_spacy_cnn/lv_spacy_cnn-1.0.0/morphologizer/cfg -> build/lib/lv_spacy_cnn/lv_spacy_cnn-1.0.0/morphologizer
copying lv_spacy_cnn/lv_spacy_cnn-1.0.0/morphologizer/model -> build/lib/lv_spacy_cnn/lv_spacy_cnn-1.0.0/morphologizer
creating build/lib/lv_spacy_cnn/lv_spacy_cnn-1.0.0/tagger
copying lv_spacy_cnn/lv_spacy_cnn-1.0.0/tagger/cfg -> build/lib/lv_

running sdist
running egg_info
writing lv_spacy_cnn.egg-info/PKG-INFO
writing dependency_links to lv_spacy_cnn.egg-info/dependency_links.txt
writing entry points to lv_spacy_cnn.egg-info/entry_points.txt
writing top-level names to lv_spacy_cnn.egg-info/top_level.txt
reading manifest file 'lv_spacy_cnn.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'lv_spacy_cnn.egg-info/SOURCES.txt'
running check
creating lv_spacy_cnn-1.0.0
creating lv_spacy_cnn-1.0.0/lv_spacy_cnn
creating lv_spacy_cnn-1.0.0/lv_spacy_cnn.egg-info
creating lv_spacy_cnn-1.0.0/lv_spacy_cnn/lv_spacy_cnn-1.0.0
creating lv_spacy_cnn-1.0.0/lv_spacy_cnn/lv_spacy_cnn-1.0.0/lemmatizer/lookups
creating lv_spacy_cnn-1.0.0/lv_spacy_cnn/lv_spacy_cnn-1.0.0/morphologizer
creating lv_spacy_cnn-1.0.0/lv_spacy_cnn/lv_spacy_cnn-1.0.0/parser
creating lv_spacy_cnn-1.0.0/lv_spacy_cnn/lv_spacy_cnn-1.0.0/senter
creating lv_spacy_cnn-1.0.0/lv_spacy_cnn/lv_spacy_cnn-1.0.0/tagger
creating lv_spacy_cnn-1.0.0/lv

In [15]:
# ==============================
# Cell 10A: install with wheel
# ==============================
import subprocess
import spacy
import pandas as pd


# install with wheel
subprocess.run([
    "pip", 
    "install", 
    "./packages/lv_spacy_cnn-1.0.0/dist/lv_spacy_cnn-1.0.0-py3-none-any.whl"
])

# load model
nlp_xlmr = spacy.load("lv_spacy_cnn")

print("lv_spacy_cnn Pipeline components:", nlp_xlmr.pipe_names)

Processing ./packages/lv_spacy_cnn-1.0.0/dist/lv_spacy_cnn-1.0.0-py3-none-any.whl
lv-spacy-cnn is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
lv_spacy_cnn Pipeline components: ['tok2vec', 'tagger', 'morphologizer', 'parser', 'senter', 'lemmatizer']


In [16]:
# ===============================
# Cell 10B: install with 'tar.gz'
# ===============================
import subprocess
import spacy
import pandas as pd

# install with '.tar.gz'
subprocess.run(["pip", "install", "./packages/lv_spacy_cnn-1.0.0/dist/lv_spacy_cnn-1.0.0.tar.gz"])
nlp_xlmr = spacy.load("lv_spacy_cnn")

print("lv_spacy_cnn Pipeline components:", nlp_xlmr.pipe_names)

Processing ./packages/lv_spacy_cnn-1.0.0/dist/lv_spacy_cnn-1.0.0.tar.gz
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  DEPRECATION: Building 'lv_spacy_cnn' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'lv_spacy_cnn'. Discussion can be found at https://github.com/pypa/pip/issues/6334


  Created wheel for lv_spacy_cnn: filename=lv_spacy_cnn-1.0.0-py3-none-any.whl size=9419711 sha256=6b0c0dee4691e9862fb328746383784c87f40cc30df86bc13ab3c9550b89b120
  Stored in directory: /home/jesse/.cache/pip/wheels/ce/6f/81/d15190767e54e54bddab684ebf05fb7278a2655603fc116368
Successfully built lv_spacy_cnn
  Attempting uninstall: lv_spacy_cnn
    Found existing installation: lv_spacy_cnn 1.0.0
    Uninstalling lv_spacy_cnn-1.0.0:
      Successfully uninstalled lv_spacy_cnn-1.0.0
lv_spacy_cnn Pipeline components: ['tok2vec', 'tagger', 'morphologizer', 'parser', 'senter', 'lemmatizer']


In [17]:
# ======================
# Cell 11A: Demo Testing
# ======================

import spacy
import numpy as np

# Load the pipeline
nlp = spacy.load("lv_spacy_cnn")

# Example text
text = """Baltijas jūras nosaukums ir devis nosaukumu baltu valodām un Baltijas valstīm.
Terminu "Baltijas jūra" (Mare Balticum) pirmoreiz lietoja vācu hronists Brēmenes Ādams 11. gadsimtā."""

# Process text
doc = nlp(text)

# ------------------------
# Tokenization 
# ------------------------
print("Tokens:")
print([token.text for token in doc])

# ------------------------
# Lemmatization / 词形还原
# ------------------------
print("Lemmas:")
print([token.lemma_ for token in doc])

# ------------------------
# Part-of-Speech Tagging
# ------------------------
print("POS tags:")
for token in doc:
    print(f"{token.text}: {token.pos_} ({token.tag_})")


# ------------------------
# Morphological Features 
# ------------------------
print("Morphological features:")
for token in doc:
    print(f"{token.text}: {token.morph}")


# ------------------------
# Dependency Parsing
# ------------------------
print("Dependency parsing: ")
for token in doc:
    print(f"{token.text} <--{token.dep_}-- {token.head.text}")


# ------------------------
# Sentence Segmentation 
# ------------------------
print("Sentences :")
for sent in doc.sents:
    print(sent.text)

# ------------------------
# Pipeline components
# ------------------------
print("Pipeline components:")
print(nlp.pipe_names)

# Tok2Vec 
vectors = np.vstack([token.vector for token in doc])
print("Token vectors shape:", vectors.shape)


Tokens:
['Baltijas', 'jūras', 'nosaukums', 'ir', 'devis', 'nosaukumu', 'baltu', 'valodām', 'un', 'Baltijas', 'valstīm', '.', '\n', 'Terminu', '"', 'Baltijas', 'jūra', '"', '(', 'Mare', 'Balticum', ')', 'pirmoreiz', 'lietoja', 'vācu', 'hronists', 'Brēmenes', 'Ādams', '11', '.', 'gadsimtā', '.']
Lemmas:
['Baltijas', 'jūra', 'nosaukums', 'būt', 'dot', 'nosaukums', 'balts', 'valoda', 'un', 'Baltijas', 'valsts', '.', '\n', 'Terminu', '"', 'Baltijas', 'jūra', '"', '(', 'Mare', 'Balticum', ')', 'pirmoreiz', 'lietot', 'vāci', 'hronists', 'Brēmenes', 'Ādams', '11', '.', 'gadsimts', '.']
POS tags:
Baltijas: PROPN (npfsg4)
jūras: NOUN (ncfsg4)
nosaukums: NOUN (ncmsn1)
ir: AUX (vcnipii30an)
devis: VERB (vmnpdmsnasnpn)
nosaukumu: NOUN (ncmpg1)
baltu: NOUN (ncmpg1)
valodām: NOUN (ncfpd4)
un: CCONJ (cc)
Baltijas: PROPN (npfsg4)
valstīm: NOUN (ncfpd6)
.: PUNCT (zs)

: VERB (affpnyp)
Terminu: ADV (npmsa1)
": PUNCT (zq)
Baltijas: PROPN (npfsg4)
jūra: NOUN (ncfsn4)
": PUNCT (zq)
(: PUNCT (zb)
Mare: X (xf

In [18]:
# ===================================================
# Cell 11B: Testing model, espically Lemma and senter
# ===================================================
import spacy
import pandas as pd

nlp = spacy.load("lv_spacy_cnn")

text = """Baltijas jūras nosaukums ir devis nosaukumu baltu valodām un Baltijas valstīm. Terminu "Baltijas jūra" (Mare Balticum) pirmoreiz lietoja vācu hronists Brēmenes Ādams 11. gadsimtā. Viņš, kā jau vietējais hronists, pierakstījis nosaukumu, kuru lietoja iedzīvotāji. Tomēr par šī vārda izcelsmi precīzu ziņu nav. Tas varēja rasties, atsaucoties uz mītisko Ziemeļeiropā it kā esošo Baltijas salu. Pastāv arī iespēja, ka Ādams no Brēmenes veidojis šo vārdu no ģermāņu vārda belt, ar kuru tiek apzīmēti vairāki Dānijas šaurumi. Cits skaidrojums — vārds cēlies no protoindoeiropiešu valodas vārda saknes *bhel, kas nozīmē ‘balts’, ‘mirdzošs’. Šī vārda sakne saglabājusies arī vairākās mūsdienu indoeiropiešu valodās, tai skaitā latviešu valodā. Vienā no senākajām kartēm, kurā attēlota mūsdienu Latvijas teritorija — Aleksandrijas zinātnieka Klaudija Ptolemaja (ap 90.—168. m. ē.) izveidotajā Austrumeiropas kartē Baltijas jūra nosaukta par "Sarmatu jūru" (MARE SARMATICVM). Citos avotos minēts nosaukums OCEANUS SARMATICUS. Sarmati bija sena klejotāju tauta, kura runājusi indoirāņu valodā un līdz 4. gadsimtam dzīvoja Austrumeiropā. Tacits to dēvēja par "Svēbu jūru" (Mare Suebicum). Vēsturē pazīstami arī citu tautu dotie Baltijas jūras nosaukumi: "Varjagu jūra", "Barbaru jūra",[2] "Vendu jūra". Tā kādā 946. gada dokumentā to dēvēja par "Rūgu jūru" (no rūgu vārda cēlies tagadējās Rīgenes salas nosaukums),[3] bet Nestora hronikas ievadā Baltijas jūra nosaukta par Varjagu jūru.[4][5] Štumpfa (Stumpf) Eiropas kartē,[6] kas izdota Cīrihē, 1548. gadā (pārkopēta no vecākas Sebastiana Minstera veidotas kartes), Baltijas jūra tiek saukta par "Vācu jūru" (Das Deutsche Meer). Arī vecākā 1450. gadā Fra Mauro zīmētajā pasaules kartē redzams nosaukums "Ģermāņu jūra" vai pat okeāns, ko parasti piedēvē Ziemeļjūrai, tomēr minētajā kartē tā nepārprotami ir arī Baltijas jūra.[7] Latvijas piekrastes iedzīvotāji mēdza atklāto jūru dēvēt par "Dižjūru", bet Rīgas līci — par "Mazo jūru" (Mazjūru). Viduslaikos Rīgas līci latīniski dēvēja par "Līvu jūru" (Mare Livonicum). Senākajās Eiropas kartēs par jūrām tika saukti arī lielākie Baltijas jūras līči. Piemēram, Mare Finonicum sive Sinus Venedicus — tagadējais Somu jūras līcis."""

doc = nlp(text)

# Generate token table
rows = []
for sent in doc.sents:
    for token in sent:
        rows.append({
            "Text": token.text,
            "Lemma": token.lemma_,
            "POS": token.pos_,
            "Dependency": token.dep_,
            "Head": token.head.text
        })

df = pd.DataFrame(rows)

pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
display(df)  

# Sentence segmentation
print("\nSentence Segmentation")
for i, sent in enumerate(doc.sents, 1):
    print(f"Sentence {i}: {sent.text}")

,Text,Lemma,POS,Dependency,Head
0,Baltijas,Baltijas,PROPN,nmod,jūras
1,jūras,jūra,NOUN,nmod,nosaukums
2,nosaukums,nosaukums,NOUN,nsubj,devis
3,ir,būt,AUX,aux,devis
4,devis,dot,VERB,ROOT,devis
5,nosaukumu,nosaukums,NOUN,nmod,valodām
6,baltu,balts,NOUN,amod,valodām
7,valodām,valoda,NOUN,iobj,devis
8,un,un,CCONJ,cc,valstīm
9,Baltijas,Baltijas,PROPN,nmod,valstīm



Sentence Segmentation
Sentence 1: Baltijas jūras nosaukums ir devis nosaukumu baltu valodām un Baltijas valstīm.
Sentence 2: Terminu "Baltijas jūra" (Mare Balticum) pirmoreiz lietoja vācu hronists Brēmenes Ādams 11. gadsimtā.
Sentence 3: Viņš, kā jau vietējais hronists, pierakstījis nosaukumu, kuru lietoja iedzīvotāji.
Sentence 4: Tomēr par šī vārda izcelsmi precīzu ziņu nav.
Sentence 5: Tas varēja rasties, atsaucoties uz mītisko Ziemeļeiropā it kā esošo Baltijas salu.
Sentence 6: Pastāv arī iespēja, ka Ādams no Brēmenes veidojis šo vārdu no ģermāņu vārda belt, ar kuru tiek apzīmēti vairāki Dānijas šaurumi.
Sentence 7: Cits skaidrojums — vārds cēlies no protoindoeiropiešu valodas vārda saknes *bhel, kas nozīmē ‘balts’, ‘mirdzošs’.
Sentence 8: Šī vārda sakne saglabājusies arī vairākās mūsdienu indoeiropiešu valodās, tai skaitā latviešu valodā.
Sentence 9: Vienā no senākajām kartēm, kurā attēlota mūsdienu Latvijas teritorija — Aleksandrijas zinātnieka Klaudija Ptolemaja (ap 90.—168.
Sen